In [1]:
import pandas as pd
from collections import defaultdict
import math
import numpy as np

In [2]:
df = pd.read_csv('train.txt', sep='\t', header=None)
df.columns =['index', 'wordtype', 'pos']

In [3]:
threshold = 0

In [4]:
trd = pd.read_csv('train.txt', sep='\t', header=None)
trd.columns =['index', 'wordtype', 'pos']

In [5]:
from collections import OrderedDict
import collections

In [15]:
dftri = df

In [6]:
c = dict(collections.Counter(trd["wordtype"]))

In [7]:
less = dict((k, v) for k, v in c.items() if v < threshold)

In [16]:
dftri['wi'] = dftri.groupby('wordtype')['wordtype'].transform('size')

In [17]:
dftri.loc[dftri.wi < threshold, 'wordtype'] = "<unk>"

In [18]:
dff = dftri.wordtype.unique()

In [19]:
len(dff)

43193

In [20]:
vocab = {}
j = 0

In [21]:
#vocab['<s>'] = 0

In [22]:
for i in range(len(dff)):
    vocab[dff[i]] = j
    j = j+1

In [23]:
words_list = dftri["wordtype"].tolist()
tag_list = dftri["pos"].tolist()

In [24]:
# words_list.insert(0, '<s>')
# tag_list.insert(0, '<s>')

In [25]:
def create_dictionaries(vocab,words_list,tag_list):

    # initialize the dictionaries using defaultdict
    emission_counts = defaultdict(int)
    transition_counts = defaultdict(int)
    tag_counts = defaultdict(int)
    
    # Initialize "prev_tag" (previous tag) with the start state, denoted by '--s--'
    prev_tag = '<s>' 
    
    # use 'i' to track the line number in the corpus
    i = 0 
    m = 0
    # Each item in the training corpus contains a word and its POS tag
    # Go through each word and its tag in the training corpus
    for k in range(len(words_list)):
        
        # Increment the word_tag count
        i += 1
        
        # get the word and tag using the get_word_tag helper function (imported from utils_pos.py)
        #word, tag = get_word_tag(word_tag,vocab)
        word = words_list[m]
        tag = tag_list[m]
        # Increment the transition count for the previous word and tag
        transition_counts[(prev_tag, tag)] += 1
        
        # Increment the emission count for the tag and word
        emission_counts[(tag, word)] += 1

        # Increment the tag count
        tag_counts[tag] += 1

        # Set the previous tag to this tag (for the next iteration of the loop)
        prev_tag = tag
        

        m = m+1
    return emission_counts, transition_counts, tag_counts

In [26]:
emission_counts, transition_counts, tag_counts = create_dictionaries(vocab,words_list,tag_list)

In [27]:
states = sorted(tag_counts.keys())

In [28]:
def create_transition_matrix(tag_counts, transition_counts):

    # Get a sorted list of unique POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Count the number of unique POS tags
    num_tags = len(all_tags)
    
    # Initialize the transition matrix 'A'
    A = np.zeros((num_tags,num_tags))
    
    # Get the unique transition tuples (previous POS, current POS)
    trans_keys = set(transition_counts.keys())
    
    ### START CODE HERE (Return instances of 'None' with your code) ### 
    
    # Go through each row of the transition matrix A
    for i in range(num_tags):
        
        # Go through each column of the transition matrix A
        for j in range(num_tags):

            # Initialize the count of the (prev POS, current POS) to zero
            count = 0
        
            # Define the tuple (prev POS, current POS)
            # Get the tag at position i and tag at position j (from the all_tags list)
            key = (all_tags[i],all_tags[j])

            # Check if the (prev POS, current POS) tuple 
            # exists in the transition counts dictionaory
            if key in trans_keys: #complete this line
                
                # Get count from the transition_counts dictionary 
                # for the (prev POS, current POS) tuple
                count = transition_counts[key]
                
            # Get the count of the previous tag (index position i) from tag_counts
            count_prev_tag = tag_counts[all_tags[i]]
            
            # Apply smoothing using count of the tuple, alpha, 
            # count of previous tag, alpha, and number of total tags
            A[i,j] = (count) / ( count_prev_tag + num_tags)

    
    return A

In [29]:
A = create_transition_matrix(tag_counts, transition_counts)

In [30]:
vocab

{'Pierre': 0,
 'Vinken': 1,
 ',': 2,
 '61': 3,
 'years': 4,
 'old': 5,
 'will': 6,
 'join': 7,
 'the': 8,
 'board': 9,
 'as': 10,
 'a': 11,
 'nonexecutive': 12,
 'director': 13,
 'Nov.': 14,
 '29': 15,
 '.': 16,
 'Mr.': 17,
 'is': 18,
 'chairman': 19,
 'of': 20,
 'Elsevier': 21,
 'N.V.': 22,
 'Dutch': 23,
 'publishing': 24,
 'group': 25,
 'Rudolph': 26,
 'Agnew': 27,
 '55': 28,
 'and': 29,
 'former': 30,
 'Consolidated': 31,
 'Gold': 32,
 'Fields': 33,
 'PLC': 34,
 'was': 35,
 'named': 36,
 'this': 37,
 'British': 38,
 'industrial': 39,
 'conglomerate': 40,
 'A': 41,
 'form': 42,
 'asbestos': 43,
 'once': 44,
 'used': 45,
 'to': 46,
 'make': 47,
 'Kent': 48,
 'cigarette': 49,
 'filters': 50,
 'has': 51,
 'caused': 52,
 'high': 53,
 'percentage': 54,
 'cancer': 55,
 'deaths': 56,
 'among': 57,
 'workers': 58,
 'exposed': 59,
 'it': 60,
 'more': 61,
 'than': 62,
 '30': 63,
 'ago': 64,
 'researchers': 65,
 'reported': 66,
 'The': 67,
 'fiber': 68,
 'crocidolite': 69,
 'unusually': 70,
 'r

In [31]:
def create_emission_matrix(tag_counts, emission_counts, vocab):

    # get the number of POS tag
    num_tags = len(tag_counts)
    
    # Get a list of all POS tags
    all_tags = sorted(tag_counts.keys())
    
    # Get the total number of unique words in the vocabulary
    num_words = len(vocab)
    
    # Initialize the emission matrix B with places for
    # tags in the rows and words in the columns
    B = np.zeros((num_tags, num_words))
    
    # Get a set of all (POS, word) tuples 
    # from the keys of the emission_counts dictionary
    emis_keys = set(list(emission_counts.keys()))
    
    
    # Go through each row (POS tags)
    for i in range(num_tags): # complete this line
        
        # Go through each column (words)
        for j,word in enumerate(vocab): # complete this line

            # Initialize the emission count for the (POS tag, word) to zero
            count = 0
                    
            # Define the (POS tag, word) tuple for this row and column
            key =  (all_tags[i], word)

            # check if the (POS tag, word) tuple exists as a key in emission counts
            if key in emis_keys: # complete this line
        
                # Get the count of (POS tag, word) from the emission_counts d
                count = emission_counts[key]
                
            # Get the count of the POS tag
            count_tag = tag_counts[all_tags[i]]
                
            # Apply smoothing and store the smoothed value 
            # into the emission matrix B for this row and column
            B[i,j] = (count) / (count_tag + num_words)

    return B

In [32]:
B = create_emission_matrix(tag_counts, emission_counts, list(vocab))

In [33]:
emis_keys = set(list(emission_counts.keys()))

In [34]:
df1 = pd.DataFrame(A, columns = list(states))

In [35]:
df1.index = list(states)

In [36]:
transit = {}
for i in (list(states)):
    for j in (list(states)):
        if df1[i][j]!=0:
            key = (i,j)
            transit[key] = df1[i][j]

In [37]:
df2 = pd.DataFrame(B, columns = list(dff))

In [38]:
df2.index = list(states)

In [39]:
emmi = {}
for i in (list(states)):
    for j in (list(dff)):
        if df2[j][i]!=0:
            key = (i,j)
            emmi[key] = df2[j][i]

In [40]:
dfdev = pd.read_csv('dev', sep='\t', header=None)
dfdev.columns =['index', 'wordtype', 'pos']

In [41]:
dfdev['wi'] = dfdev.groupby('wordtype')['wordtype'].transform('size')

In [42]:
dfdev.loc[dfdev.wi < threshold, 'wordtype'] = "<unk>"

In [43]:
dfdev = dfdev.drop('wi', 1)

<ipython-input-43-8c9f63739cc7>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfdev = dfdev.drop('wi', 1)


In [44]:
dfdevtu = [tuple(x) for x in dfdev.to_numpy()]

In [45]:
cc1 = 0
for i in range(len(dfdevtu)):
    if dfdevtu[i][0]==1:
        cc1 = cc1+1

In [46]:
L2=list(dfdevtu)
temp_tup1 = []
final_tup1 = []
for i in range(cc1):
    temp_tup1 = []
    temp_tup1.append(L2[0])
    L2.pop(0)
    for j in range(len(L2)):
        if L2[0][0]==1:
            break
        else:
            temp_tup1.append(L2[0])
            #print(L2[0])
            L2.pop(0)
    final_tup1.append(temp_tup1)

In [47]:
outer1 = []
inner1 = []
for i in range(len(final_tup1)):
    endl1 = list(final_tup1[i])
    inner1 = []
    for j in range(len(endl1)):
        templ1 = list(endl1[j])
        templ1.pop(0)
        inner1.append(templ1)
    outer1.append(inner1)

In [48]:
wordtotagdic = dict(zip(dftri.wordtype, dftri.pos))

In [49]:
import operator
from sklearn.metrics import accuracy_score

In [73]:
def greedy_hmm(outer1, emmi, transit, wordtotagdic):
    predicted_tags = []
    for i in (outer1):
        prev_tag = "<s>"
        actual_tag = []
        for j in (i):
            res = {}
            if j == "":
                prev_tag = "<s>"
            if j[0] == "":
                continue
            if j[0] != "":
                #actual_tag.append(j[1])
                word = j[0]
                if word in wordtotagdic:
                    word = word
                else:
                    word = "the"
                value = wordtotagdic[word]
                key = (value,word)
                emission_prob = emmi[key]
                trans_key = (prev_tag, key[0])
                if trans_key in transit:
                        trans_prob = transit[trans_key]
                else:
                    trans_prob = 0
                res[(word,trans_key)] = emission_prob * trans_prob
            prev_tag = (max(res.items(), key = operator.itemgetter(1))[0])[1][1]
            predicted_tags.append(prev_tag)
    #res = accuracy_score(actual_tag,predicted_tags)*100
    #print(res)
    return actual_tag,predicted_tags

In [51]:
myhmm_actual, myhmm_pred = greedy_hmm(outer1, emmi, transit, wordtotagdic)

In [69]:
outer1

[[['The', 'DT'],
  ['Arizona', 'NNP'],
  ['Corporations', 'NNP'],
  ['Commission', 'NNP'],
  ['authorized', 'VBD'],
  ['an', 'DT'],
  ['11.5', 'CD'],
  ['%', 'NN'],
  ['rate', 'NN'],
  ['increase', 'NN'],
  ['at', 'IN'],
  ['Tucson', 'NNP'],
  ['Electric', 'NNP'],
  ['Power', 'NNP'],
  ['Co.', 'NNP'],
  [',', ','],
  ['substantially', 'RB'],
  ['lower', 'JJR'],
  ['than', 'IN'],
  ['recommended', 'VBN'],
  ['last', 'JJ'],
  ['month', 'NN'],
  ['by', 'IN'],
  ['a', 'DT'],
  ['commission', 'NN'],
  ['hearing', 'NN'],
  ['officer', 'NN'],
  ['and', 'CC'],
  ['barely', 'RB'],
  ['half', 'PDT'],
  ['the', 'DT'],
  ['rise', 'NN'],
  ['sought', 'VBN'],
  ['by', 'IN'],
  ['the', 'DT'],
  ['utility', 'NN'],
  ['.', '.']],
 [['The', 'DT'],
  ['ruling', 'NN'],
  ['follows', 'VBZ'],
  ['a', 'DT'],
  ['host', 'NN'],
  ['of', 'IN'],
  ['problems', 'NNS'],
  ['at', 'IN'],
  ['Tucson', 'NNP'],
  ['Electric', 'NNP'],
  [',', ','],
  ['including', 'VBG'],
  ['major', 'JJ'],
  ['write-downs', 'NNS'],
  [

In [52]:
act = []
for i in range(len(dfdevtu)):
    act.append(dfdevtu[i][2])

In [53]:
print(accuracy_score(act,myhmm_pred)*100)

89.18629712828607


In [54]:
list_of_dev = dfdev['wordtype'].to_list()

In [55]:
neetr = dftri.drop(['index','wi'], 1)

<ipython-input-55-b9b8d1c41d92>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  neetr = dftri.drop(['index','wi'], 1)


In [56]:
netrl = neetr.values.tolist()

In [57]:
def Vanilla_Viterbi(words, train_bag = netrl):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    for key, word in enumerate(words):

        #initialise list of probability column for a given observation
        p = [] 
        p_transition =[] # list for storing transition probabilities
        for tag in T:
            if key == 0:
                transition_p = df1.loc['.', tag]
            else:
                transition_p = df1.loc[state[-1], tag]
                
            # compute emission and state probabilities
            #print(word)
            key1 = (tag,word)
            if key1 in emmi:
                emission_p = df2.loc[tag,word]
            else:
                emission_p = 0
            #emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            #print(emission_p)
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            p_transition.append(transition_p)
            
        pmax = max(p)
        state_max = T[p.index(pmax)] 
        
      
        # if probability is zero (unknown word) then use transition probability
        if(pmax==0):
            pmax = max(p_transition)
            state_max = T[p_transition.index(pmax)]
                           
        else:
            state_max = T[p.index(pmax)] 
        
        state.append(state_max)
    return list(zip(words, state))

In [58]:
tagged_seq = Vanilla_Viterbi(list_of_dev)

In [59]:
dfdevtag = dfdev.drop('index', 1)

<ipython-input-59-6358e360c42b>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dfdevtag = dfdev.drop('index', 1)


In [60]:
tagch = dfdevtag.values.tolist()

In [61]:
samp = []
for i in range(len(tagch)):
    samp.append(tuple(tagch[i]))

In [62]:
check = [i for i, j in zip(tagged_seq, samp) if i == j] 

In [63]:
accuracy = len(check)/len(tagged_seq)

In [64]:
print('Viterbi Algorithm Accuracy: ',accuracy*100)

Viterbi Algorithm Accuracy:  93.30262279157307


In [65]:
dtest = pd.read_csv('test', sep='\t', header=None)
#dtest.columns =['index', 'wordtype', 'pos']

In [67]:
dtest.columns =['index', 'wordtype']

In [92]:
dtestlis

['Influential',
 'members',
 'of',
 'the',
 'House',
 'Ways',
 'and',
 'Means',
 'Committee',
 'introduced',
 'legislation',
 'that',
 'would',
 'restrict',
 'how',
 'the',
 'new',
 'savings-and-loan',
 'bailout',
 'agency',
 'can',
 'raise',
 'capital',
 ',',
 'creating',
 'another',
 'potential',
 'obstacle',
 'to',
 'the',
 'government',
 "'s",
 'sale',
 'of',
 'sick',
 'thrifts',
 '.',
 'The',
 'bill',
 ',',
 'whose',
 'backers',
 'include',
 'Chairman',
 'Dan',
 'Rostenkowski',
 '(',
 'D.',
 ',',
 'Ill.',
 ')',
 ',',
 'would',
 'prevent',
 'the',
 'Resolution',
 'Trust',
 'Corp.',
 'from',
 'raising',
 'temporary',
 'working',
 'capital',
 'by',
 'having',
 'an',
 'RTC-owned',
 'bank',
 'or',
 'thrift',
 'issue',
 'debt',
 'that',
 'would',
 "n't",
 'be',
 'counted',
 'on',
 'the',
 'federal',
 'budget',
 '.',
 'The',
 'bill',
 'intends',
 'to',
 'restrict',
 'the',
 'RTC',
 'to',
 'Treasury',
 'borrowings',
 'only',
 ',',
 'unless',
 'the',
 'agency',
 'receives',
 'specific',
 '

In [70]:
dtestlis = dtest['wordtype'].values.tolist()

In [93]:
myhmm_testac, myhmm_testpre = greedy_hmm(dtestlis, emmi, transit, wordtotagdic)

In [78]:
viter_test

[('Influential', 'DT'),
 ('members', 'NNS'),
 ('of', 'IN'),
 ('the', 'DT'),
 ('House', 'NNP'),
 ('Ways', 'NNP'),
 ('and', 'CC'),
 ('Means', 'NNP'),
 ('Committee', 'NNP'),
 ('introduced', 'VBD'),
 ('legislation', 'NN'),
 ('that', 'IN'),
 ('would', 'MD'),
 ('restrict', 'VB'),
 ('how', 'WRB'),
 ('the', 'DT'),
 ('new', 'JJ'),
 ('savings-and-loan', 'NN'),
 ('bailout', 'NN'),
 ('agency', 'NN'),
 ('can', 'MD'),
 ('raise', 'VB'),
 ('capital', 'NN'),
 (',', ','),
 ('creating', 'VBG'),
 ('another', 'DT'),
 ('potential', 'JJ'),
 ('obstacle', 'NN'),
 ('to', 'TO'),
 ('the', 'DT'),
 ('government', 'NN'),
 ("'s", 'POS'),
 ('sale', 'NN'),
 ('of', 'IN'),
 ('sick', 'JJ'),
 ('thrifts', 'NNS'),
 ('.', '.'),
 ('The', 'DT'),
 ('bill', 'NN'),
 (',', ','),
 ('whose', 'WP$'),
 ('backers', 'NNS'),
 ('include', 'VBP'),
 ('Chairman', 'NNP'),
 ('Dan', 'NNP'),
 ('Rostenkowski', 'NNP'),
 ('(', '-LRB-'),
 ('D.', 'NNP'),
 (',', ','),
 ('Ill.', 'NNP'),
 (')', '-RRB-'),
 (',', ','),
 ('would', 'MD'),
 ('prevent', 'VB'),

In [76]:
viter_test = Vanilla_Viterbi(dtestlis)

In [ ]:
dtest

In [80]:
ans = []
for i in range(len(viter_test)):
    ans.append(viter_test[i][1])

In [81]:
ans

['DT',
 'NNS',
 'IN',
 'DT',
 'NNP',
 'NNP',
 'CC',
 'NNP',
 'NNP',
 'VBD',
 'NN',
 'IN',
 'MD',
 'VB',
 'WRB',
 'DT',
 'JJ',
 'NN',
 'NN',
 'NN',
 'MD',
 'VB',
 'NN',
 ',',
 'VBG',
 'DT',
 'JJ',
 'NN',
 'TO',
 'DT',
 'NN',
 'POS',
 'NN',
 'IN',
 'JJ',
 'NNS',
 '.',
 'DT',
 'NN',
 ',',
 'WP$',
 'NNS',
 'VBP',
 'NNP',
 'NNP',
 'NNP',
 '-LRB-',
 'NNP',
 ',',
 'NNP',
 '-RRB-',
 ',',
 'MD',
 'VB',
 'DT',
 'NNP',
 'NNP',
 'NNP',
 'IN',
 'VBG',
 'JJ',
 'NN',
 'NN',
 'IN',
 'VBG',
 'DT',
 'NN',
 'NN',
 'CC',
 'NN',
 'NN',
 'NN',
 'IN',
 'MD',
 'RB',
 'VB',
 'VBN',
 'IN',
 'DT',
 'JJ',
 'NN',
 '.',
 'DT',
 'NN',
 'VBZ',
 'TO',
 'VB',
 'DT',
 'NNP',
 'TO',
 'NNP',
 'NNS',
 'RB',
 ',',
 'IN',
 'DT',
 'NN',
 'VBZ',
 'JJ',
 'JJ',
 'NN',
 '.',
 '``',
 'JJ',
 'NN',
 '``',
 'PRP',
 'POS',
 'NN',
 'VBZ',
 'JJ',
 'CC',
 'JJ',
 ',',
 'RB',
 'JJR',
 'JJ',
 'IN',
 'JJ',
 'NNP',
 'NN',
 ',',
 "''",
 'VBD',
 'NNP',
 'NNP',
 'NNP',
 '-LRB-',
 'NNP',
 ',',
 'NNP',
 '-RRB-',
 ',',
 'DT',
 'NN',
 'POS',
 'JJ',


In [86]:
dviter = dtest

In [90]:
dmyhmm = dtest

In [87]:
dviter

,index,wordtype
0,1,Influential
1,2,members
2,3,of
3,4,the
4,5,House
...,...,...
129649,26,them
129650,27,here
129651,28,with
129652,29,us


In [112]:
dtest

,index,wordtype,predicted_viterbi
0,1,Influential,DT
1,2,members,NNS
2,3,of,IN
3,4,the,DT
4,5,House,NNP
...,...,...,...
129649,26,them,PRP
129650,27,here,RB
129651,28,with,IN
129652,29,us,PRP


In [88]:
dviter['predicted_viterbi'] = ans

In [89]:
dviter

,index,wordtype,predicted_viterbi
0,1,Influential,DT
1,2,members,NNS
2,3,of,IN
3,4,the,DT
4,5,House,NNP
...,...,...,...
129649,26,them,PRP
129650,27,here,RB
129651,28,with,IN
129652,29,us,PRP


In [94]:
len(myhmm_testpre)

573736

In [102]:
hymmt = [tuple(x) for x in dtest.to_numpy()]

In [104]:
cc1 = 0
for i in range(len(hymmt)):
    if hymmt[i][0]==1:
        cc1 = cc1+1

L2=list(hymmt)
temp_tup1 = []
final_tup1 = []
for i in range(cc1):
    temp_tup1 = []
    temp_tup1.append(L2[0])
    L2.pop(0)
    for j in range(len(L2)):
        if L2[0][0]==1:
            break
        else:
            temp_tup1.append(L2[0])
            #print(L2[0])
            L2.pop(0)
    final_tup1.append(temp_tup1)

outer1 = []
inner1 = []
for i in range(len(final_tup1)):
    endl1 = list(final_tup1[i])
    inner1 = []
    for j in range(len(endl1)):
        templ1 = list(endl1[j])
        templ1.pop(0)
        inner1.append(templ1)
    outer1.append(inner1)

In [106]:
myhmm_actual, myhmm_pred = greedy_hmm(outer1, emmi, transit, wordtotagdic)

In [107]:
len(myhmm_pred)

129654

In [110]:
dhmmtest = dtest.drop('predicted_viterbi', 1)

<ipython-input-110-075ad380f3fb>:1: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  dhmmtest = dtest.drop('predicted_viterbi', 1)


In [111]:
dhmmtest

,index,wordtype
0,1,Influential
1,2,members
2,3,of
3,4,the
4,5,House
...,...,...
129649,26,them
129650,27,here
129651,28,with
129652,29,us


In [113]:
dhmmtest['predicted_hmm'] = myhmm_pred

In [116]:
dhmmtest.to_csv(r'greedy.out', header=None, index=None, sep=' ', mode='a')

In [117]:
dviter.to_csv(r'viterbi.out', header=None, index=None, sep=' ', mode='a')

In [126]:
import simplejson as json
f = open("hmm.json", "w")
json.dump(transition, f)
json.dump(emission, f)
f.close()

In [124]:
emission = {}
for i in (emmi):
    emission[str(i)] = emmi[i]

In [125]:
transition = {}
for i in (transit):
    transition[str(i)] = transit[i]

In [127]:
df_new.to_csv(r'vocabulary.txt', header=None, index=None, sep=' ', mode='a')